In [73]:
import numpy as np
# import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline

import requests as re

sns.set_context('poster')
sns.set_style('whitegrid')


from sklearn.tree import export_graphviz


#Trees
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

#Preprocessing packages
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer #One hot encoding
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

#Cross Validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

#Metrics
from sklearn.metrics import confusion_matrix, auc, roc_curve

import seaborn as sns
import matplotlib.pyplot as plt
import pydotplus 
from IPython.display import Image 

#Bootstrap
from sklearn.utils import resample

%matplotlib inline

This file explores the NLP used to classify listings based on the type of home. In this file, we first split up the dataset (that was successfully classified) into many smaller datasets based on type (partition the data), and then train a Random Forest Classifier on each partition. The classification accuracy for each was then compared to two more models. The first model it is compared to is a basic Random Forest Classifier trained without 'typed' data. The second is a Random Forest Classifier trained with all the 'typed' data (binary columns of type). The results of partitioned Forests were then compared, to see if a partitioned Random Forest performed better than the others. I.e. are the features that are important for listing interest different for a different type of home?

In [125]:
train = pd.read_json("typedData/train_typed.json")
test = pd.read_json("typedData/testX.json")

In [126]:
# Split data by type

train_orig = train.copy()

y_train = train['interest_level']

y_train_stud = train[train['studio'] == 1]['interest_level']
y_train_town = train[train['townhome'] == 1]['interest_level']
y_train_appt = train[train['apartment'] == 1]['interest_level']
y_train_loft = train[train['loft'] == 1]['interest_level']
y_train_cond = train[train['condominium'] == 1]['interest_level']
y_train_ph = train[train['ph'] == 1]['interest_level']

train = train.drop(['interest_level','interestVal','description','display_address','created','building_id','features', 'manager_id', 'photos', 'street_address'], axis=1)

x_train_stud = train[train['studio'] == 1]
x_train_town = train[train['townhome'] == 1]
x_train_appt = train[train['apartment'] == 1]
x_train_loft = train[train['loft'] == 1]
x_train_cond = train[train['condominium'] == 1]
x_train_ph = train[train['ph'] == 1]

# Tree for studio, townhome, apartment, condominium, loft, pd

In [127]:
y_test_stud = test[test['studio'] == 1]['interest_level']
y_test_town = test[test['townhome'] == 1]['interest_level']
y_test_appt = test[test['apartment'] == 1]['interest_level']
y_test_loft = test[test['loft'] == 1]['interest_level']
y_test_cond = test[test['condominium'] == 1]['interest_level']
y_test_ph = test[test['ph'] == 1]['interest_level']

y_test = test['interest_level']
test = test.drop(['interest_level','description','display_address','created','building_id','features', 'manager_id', 'photos', 'street_address'], axis=1)


x_test_stud = test[test['studio'] == 1]
x_test_town = test[test['townhome'] == 1]
x_test_appt = test[test['apartment'] == 1]
x_test_loft = test[test['loft'] == 1]
x_test_cond = test[test['condominium'] == 1]
x_test_ph = test[test['ph'] == 1]

In [142]:
# Random Forest on Studio Listings
sqrt_forest = RandomForestClassifier(warm_start=True, 
                             oob_score=True,
                             max_features="sqrt",
                             random_state = 1)

sqrt_forest.set_params(n_estimators=170)
sqrt_forest.fit(x_train_stud, y_train_stud)

print("The score for a tree trained with studio listings only (and predicted for studio listings) is: {} ".format(sqrt_forest.score(x_test_stud, y_test_stud)))
print(sqrt_forest.oob_score_)

The score for a tree trained with studio listings only (and predicted for studio listings) is: 0.6911595866819747 
0.700574712644


In [144]:
# Townhouse listings only
sqrt_forest = RandomForestClassifier(warm_start=True, 
                             oob_score=True,
                             max_features="sqrt",
                             random_state = 1)

sqrt_forest.set_params(n_estimators=170)
sqrt_forest.fit(x_train_town, y_train_town)

print("The score for a tree trained and tested with townhouse listings only is: {} ".format(sqrt_forest.score(x_test_town, y_test_town)))
print(sqrt_forest.oob_score_)

The score for a tree trained and tested with townhouse listings only is: 0.71875 
0.659751037344


In [145]:
# Apartment Listings Only
sqrt_forest = RandomForestClassifier(warm_start=True, 
                             oob_score=True,
                             max_features="sqrt",
                             random_state = 1)

sqrt_forest.set_params(n_estimators=170)
sqrt_forest.fit(x_train_appt, y_train_appt)

print("The score for a tree trained and tested with apartment listings only is: {}".format(sqrt_forest.score(x_test_appt, y_test_appt)))
print(sqrt_forest.oob_score_)


# Tree for studio, townhome, apartment, condominium, loft, pd

The score for a tree trained and tested with apartment listings only is: 0.7007227734901385
0.694956875898


In [146]:
# Condominium Listings Only
sqrt_forest = RandomForestClassifier(warm_start=True, 
                             oob_score=True,
                             max_features="sqrt",
                             random_state = 1)

sqrt_forest.set_params(n_estimators=170)
sqrt_forest.fit(x_train_cond, y_train_cond)

print("The score for a tree trained and tested with condominium listings only is: {}.".format(sqrt_forest.score(x_test_cond, y_test_cond)))
print(sqrt_forest.oob_score_)

The score for a tree trained and tested with condominium listings only is: 0.7426470588235294.
0.671875


In [148]:
# Loft listings only
sqrt_forest = RandomForestClassifier(warm_start=True, 
                             oob_score=True,
                             max_features="sqrt",
                             random_state = 1)

sqrt_forest.set_params(n_estimators=170)
sqrt_forest.fit(x_train_loft, y_train_loft)
print("The score for a tree trained and tested with loft listings only is: {}.".format(sqrt_forest.score(x_test_loft, y_test_loft)))
print(sqrt_forest.oob_score_)

The score for a tree trained and tested with loft listings only is: 0.6470588235294118.
0.769230769231


In [151]:
# Penthouse listings only
sqrt_forest = RandomForestClassifier(warm_start=True, 
                             oob_score=True,
                             max_features="sqrt",
                             random_state = 1)

sqrt_forest.set_params(n_estimators=170)
sqrt_forest.fit(x_train_ph, y_train_ph)

print("The score for a tree trained and tested with penthouse listings only is: {}.".format(sqrt_forest.score(x_test_ph, y_test_ph)))
print(sqrt_forest.oob_score_)

The score for a tree trained and tested with penthouse listings only is: 0.6334661354581673.
0.698473282443


In [120]:
# First, investigate differences in loft and Loft
loft_descrepancies = train['loft'] == train['Loft']

num_desc = [True for t in loft_descrepancies if t == False]
print(len(num_desc))
print(len(num_desc)/len(train['loft']))

brad_loft_num = train[train['loft'] == 1]
print(len(brad_loft_num))
print(len(brad_loft_num[brad_loft_num['Loft'] == 1]))

given_loft_num = len(train[train['Loft'] == 1])

print(given_loft_num)

1088
0.05499671435070515
156
28
988


In [150]:
train_untyped = train.drop(['loft','ph','condominium','apartment','townhome','studio'],axis=1)
test_untyped = test.drop(['loft','ph','condominium','apartment','townhome','studio'],axis=1)

In [129]:
# Tree WITHOUT type labels
sqrt_forest = RandomForestClassifier(warm_start=True, 
                             oob_score=True,
                             max_features="sqrt",
                             random_state = 1)

sqrt_forest.set_params(n_estimators=170)
sqrt_forest.fit(train_untyped, y_train)

print(sqrt_forest.score(test_untyped,y_test))
print(sqrt_forest.oob_score_)

0.704258594151
0.698781782338


In [133]:
# single tree including type labels
gen_forest = RandomForestClassifier(warm_start=True, 
                             oob_score=True,
                             max_features="sqrt",
                             random_state = 1)

gen_forest.set_params(n_estimators=170)
gen_forest.fit(train, y_train)

print(gen_forest.score(test,y_test))
print(gen_forest.oob_score_)

0.703027193433
0.695091745438


In [ ]:
# The true indicator is seeing the difference in score for the 
# combined typed tree and the partitioned typed tree

In [135]:
#Partitioned Studio score: 0.6911595866819747 
print(gen_forest.score(x_test_stud,y_test_stud))

0.712973593571


In [134]:
#Partitioned Townhome score: 0.71875 
print(gen_forest.score(x_test_town,y_test_town))

0.71875


In [136]:
#Partitioned apartment score: 0.7007227734901385
print(gen_forest.score(x_test_appt, y_test_appt))

0.702437829229


In [137]:
#Partitioned condominium score: 0.742647058823529
print(gen_forest.score(x_test_cond, y_test_cond))

0.75


In [139]:
# Partitioned Loft score: 0.6470588235294118
print(gen_forest.score(x_test_loft, y_test_loft))

0.735294117647


In [140]:
# Partitioned Penthouse score: 0.6334661354581673
print(gen_forest.score(x_test_ph, y_test_ph))

0.641434262948


In [152]:
# Notable changes: Condominium increased with general tree, Loft increased with general tree
# Minor changes: penthouse, loft, apartment did better on general tree by a very small amount

Conclusion: housing type does not seem to play a significant role in creating a better model to predict interest level, nor does it seem to play a significant role in interest level at all.